In [1]:
!pip install transformers

In [2]:
!wandb off

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [ ]:
# import wandb
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient() 

# personal_key_for_api = user_secrets.get_secret("wandb-key")

# ! wandb login $personal_key_for_api
# wandb.init()

In [3]:
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [4]:
# x=tt['tweet'][1]
# text3=x.split()
# text2=' '.join(text3[:len(text3)//2])
# text2

In [5]:
# x[x.index(text2) + len(text2):]

In [6]:
tt=pd.read_csv("/kaggle/input/mahasent/tweets-valid.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("tweets-valid.csv",index=False)

In [7]:
tt=pd.read_csv("/kaggle/input/mahasent/tweets-test.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("tweets-test.csv",index=False)

In [8]:
tt=pd.read_csv("/kaggle/input/bedata/MahaSent/mahasent_train_new_part1.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("MahaSent.csv")

In [8]:
# dataset class
class SentimentDataset(Dataset):
  def __init__(self, txt_list, label_list, tokenizer, max_length):
    # define variables
    self.input_ids = []
    self.attn_masks = []
    self.labels = []
    map_label = {-1:0, 0:1, 1:2}

    # iterate through the dataset
    for txt, label in zip(txt_list, label_list):
      # prepare the text
      prep_txt = f'<startoftext>{map_label[label]}<sep>{txt}<endoftext>'
      # tokenize
      encodings_dict = tokenizer(prep_txt, truncation=True, max_length=max_length, padding='max_length')
      # append to list
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
      self.labels.append(map_label[label])

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

In [9]:
# Data load function
def load_sentiment_dataset(tokenizer):
  # load dataset and sample all reviews in the dataset
  file_path = 'MahaSent.csv'
  df = pd.read_csv(file_path)
  df = df.sample(len(df), random_state=1)
#   test_df=pd.read_csv("")

  # divide into test and train
#   X_train, X_test, y_train, y_test = train_test_split(df['tweet'].tolist(), df['label'].tolist(), shuffle=True, test_size=0.05, random_state=1, stratify=df['label'])
  X_train=df['tweet'][:]
  X_test=df['tweet'][:]
  y_train=df['label'][:]
  y_test=df['label'][:]
  # format into SentimentDataset class
  train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=512)

  # return
  return train_dataset, (X_test, y_test)

In [10]:
model_name = 'l3cube-pune/marathi-gpt'

# seed
torch.manual_seed(42)

# load tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-gpt", bos_token='<startoftext>', eos_token = '<endoftext>', pad_token='<pad>', unk_token='<sep>')
model = AutoModelForCausalLM.from_pretrained("l3cube-pune/marathi-gpt").cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50261, 768)

In [11]:
# prepare and load dataset
train_dataset, test_dataset = load_sentiment_dataset(tokenizer)

In [12]:
# TRAIN
# creating training arguments
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir='results', num_train_epochs=3, logging_steps=10, save_strategy="epoch", per_device_train_batch_size=2,
                                  per_device_eval_batch_size=2, warmup_steps=100, weight_decay=0.01, logging_dir='logs')

In [ ]:
# start training
from transformers import Trainer
Trainer(model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=lambda data: {
            'input_ids':torch.stack([f[0] for f in data]),
            'attention_mask':torch.stack([f[1] for f in data]),
            'labels':torch.stack([f[0] for f in data])
        }).train()

In [ ]:
!zip -r checkpoint.zip results/checkpoint-15145

In [ ]:
# !pip install GPUtil

# from GPUtil import showUtilization as gpu_usage
# gpu_usage()

In [ ]:
# !pip install GPUtil

# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()

# free_gpu_cache()  

In [ ]:
# TEST

# set the model to eval mode
_ = model.eval()

# run model inference on all test data
original_label, predicted_label, original_text, predicted_text = [], [], [], []
map_label = {-1:0, 0:1, 1:2}

# iterate over all test data
from tqdm import tqdm
import re
for text, label in tqdm(zip(test_dataset[0], test_dataset[1])):
  # create prompt (in compliance with the one used during training)
  text3=text.split()
  text2=' '.join(text3[:len(text3)//2])
  prompt = f'<startoftext>{label}<sep>{text2}'
  # generate tokens
  generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
  # perform prediction
  sample_outputs = model.generate(generated, do_sample=False, top_k=50, 
                                  max_length=512, top_p=0.9, temperature=0, 
                                  num_return_sequences=0)
  # decode the predicted tokens into texts
  pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  # extract the predicted sentiment
  print("A:",prompt)
  print("G:",pred_text)
  try:
    pred_sentiment = re.findall("<sep>(.*)", pred_text)[-1]
  except:
    pred_sentiment="None"

  # append results
  original_label.append(map_label[label])
  predicted_label.append(pred_sentiment)
  original_text.append(text)
  predicted_text.append(pred_text)

In [17]:
# TEST

# set the model to eval mode
_ = model.eval()

# run model inference on all test data
original_label, predicted_label, original_text, predicted_text = [], [], [], []
map_label = {-1:0, 0:1, 1:2}

# iterate over all test data
from tqdm import tqdm
import re
prompt=[]
final=[]
x=1
for text, label in tqdm(zip(test_dataset[0], test_dataset[1])):
    text3=text.split()
    text2=' '.join(text3[:len(text3)//2])
    prompt.append(f'<startoftext>{label}<sep>{text2}')
    original_label.append(map_label[label])
    predicted_label.append(prompt)
    original_text.append(text)
    predicted_text.append(prompt)
#     print(x," ",x%1000)
    if x%100==0:     
        print(x)
        generated = tokenizer(prompt, return_tensors="pt",padding=True,truncation=True).to("cuda")
        # perform prediction
        with torch.no_grad():
            sample_outputs = model.generate(**generated, do_sample=False, top_k=50, 
                                      max_length=512, top_p=0.9, temperature=0, 
                                      num_return_sequences=0).cuda()
        # decode the predicted tokens into texts
        pred_text = tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)
        # extract the predicted sentiment
#         print("A:",prompt)
#         print("G:",pred_text)
        final.extend(pred_text)
        prompt=[]
    x+=1
# print(final)
final.extend(prompt)
# # append results


0it [00:00, ?it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100


100it [00:24,  4.08it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


200


200it [00:46,  4.33it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

5800it [20:50,  4.66it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


5900


5900it [21:11,  4.66it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


6000


6056it [21:33,  4.68it/s]


In [26]:
final=final[:6000]

In [27]:
generated = tokenizer(prompt, return_tensors="pt",padding=True,truncation=True).to("cuda")
        # perform prediction
with torch.no_grad():
    sample_outputs = model.generate(**generated, do_sample=False, top_k=50, max_length=512, top_p=0.9, temperature=0, num_return_sequences=0).cuda()
            # decode the predicted tokens into texts
    pred_text = tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [28]:
final.extend(pred_text)

In [32]:
final[-1]

'1 श्री संत तुकाराम महाराज बीज उत्सव मराठी भक्ती परंपरेत अनन्यसाधारण स्थान असलेले संत तुकाराम महाराज यांनी संसार ातील सर्व सुख - दुः खे परखड पणे करून, त्यांच्या कार्याचा ठसा उमट विला.'

In [36]:
y=test_dataset[1].reset_index(drop=True)

In [50]:
x=test_dataset[0].reset_index(drop=True)

In [38]:
x

0       नूतन वर्ष २०१९ सुख, समृद्धीचे आणि भरभराटीचे जा...
1       कंगना बाई ही मोदी सरकारप्रमाणेच पूर्णपणे खोटार...
2       घरी रहा, सुरक्षित रहा...\nआरोग्य सेतू ॲप आपल्य...
3       दरम्यान जिगाव प्रकल्पाच्या पाणीसाठ्याबाबत, पुन...
4       आचारसंहिता लागू झाली. सरकारच्या प्रत्येक विभाग...
                              ...                        
6051    गत ७० वर्षांच्या कालावधीत पहिल्यांदाच डिझेल पे...
6052    कोल्हापूरची अंबाबाई, कोल्हापूरचा तांबडा-पांढरा...
6053    हे प्रधानमंत्री गरीब कल्याण पॅकेज नसून \n" प्र...
6054    महिलांना त्यांच्या हक्काची,आत्मनिर्भरतेची जाणी...
6055    श्री संत तुकाराम महाराज बीज उत्सव मराठी भक्तीप...
Name: tweet, Length: 6056, dtype: object

In [40]:
kai=pd.DataFrame({'label':y,'original':x,'generated':final})    

In [ ]:
# # TEST

# # set the model to eval mode
# _ = model.eval()

# # run model inference on all test data
# original_label, predicted_label, original_text, predicted_text = [], [], [], []
# map_label = {-1:0, 0:1, 1:2}

# # iterate over all test data
# from tqdm import tqdm
# import re
# for text, label in tqdm(zip(test_dataset[0], test_dataset[1])):
#   # create prompt (in compliance with the one used during training)
#   text2=text[:len(text)//2]
#   prompt = f'<startoftext>{label}<sep>{text2}'
#   # generate tokens
#   generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
#   # perform prediction
#   sample_outputs = model.generate(generated, do_sample=False, top_k=50, 
#                                   max_length=512, top_p=0.9, temperature=0, 
#                                   num_return_sequences=0).cuda()
#   # decode the predicted tokens into texts
#   pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
#   # extract the predicted sentiment
#   print("A:",prompt)
#   print("G:",pred_text)
#   try:
#     pred_sentiment = re.findall("<sep>(.*)", pred_text)[-1]
#   except:
#     pred_sentiment="None"

#   # append results
#   original_label.append(map_label[label])
#   predicted_label.append(pred_sentiment)
#   original_text.append(text)
#   predicted_text.append(pred_text)

In [ ]:
# len(final)

In [41]:
kai=pd.DataFrame({"original_text":original_text,"predicted_text":predicted_text})
kai.to_csv("Predictions.csv")

In [ ]:
df1 = pd.DataFrame({
    'original_text':original_text,
    'original_label':original_label,
    'predicted_label': predicted_label,
    'predicted_text': predicted_text
})
df1.to_csv('myCsv.csv', index=False)

In [ ]:
kai

In [ ]:
# predicted_label=[]
# for i in range(len(final)):
#     if(final[i].split()[0]=='-'):
#         predicted_label.append(-1)
#     else:
#         if(len(final[i].split()[0])>1):
#             try:
#                 predicted_label.append(int(final[i].split()[0][0]))
#             except:
#                 if (final[i].split()[0][13:14]=='-'):
#                     predicted_label.append(-1)
#                 elif final[i].split()[0][13:14]=='0':
#                     predicted_label.append(0)
#                 else:
#                     predicted_label.append(1)
#         else:
#             predicted_label.append(int(final[i].split()[0]))

In [ ]:
# predicted_label=[]
# gg=[]
# for i in range(len(final)):
#     if(final[i].split()[0]=='-'):
#         predicted_label.append(-1)
#         gg.append(' '.join(final[i].split()[2:]))
#     else:
#         try:
#             predicted_label.append(int(final[i].split()[0]))
#             gg.append(' '.join(final[i].split()[2:]))
#         except:
#             print(final[i])

In [ ]:
# predicted_label

In [ ]:
# df_valid = pd.read_csv('tweets-valid.csv')
# df_valid

In [ ]:
# df_valid['tweet'][1]

In [ ]:
# df_valid['tweet'][2][:len(df_valid['tweet'][1])//2]

In [ ]:
# prompt = '<startoftext>'+str(df_valid['label'][2])+'<sep>'+df_valid['tweet'][1][:len(df_valid['tweet'][2])//2]
#   # generate tokens
# generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
#   # perform prediction
# sample_outputs = model.generate(generated, do_sample=False, top_k=50, 
#                                   max_length=512, top_p=0.9, temperature=0, 
#                                   num_return_sequences=0)
#   # decode the predicted tokens into texts
# pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)

In [ ]:
# pred_text

## Combining both datasets

In [42]:
preds=pd.read_csv("/kaggle/working/Predictions.csv")
predicted_text=preds["generated"]

In [43]:
# predicted_text=predicted_text[:12100]

In [44]:
augmented_label=[]
augmented_text=[]
for i in range(len(predicted_text)):
  x=predicted_text[i].split()
  if x[0]=="-":
    augmented_label.append(-1)  
    augmented_text.append(' '.join(x[2:]))
  else:
    try:
      augmented_label.append(int(x[0]))
    except:
      augmented_label.append(int(x[0][0]))
    augmented_text.append(' '.join(x[1:]))

In [49]:
len(x)

32

In [51]:
gg=pd.DataFrame({'label':y,'original':x,'generated':augmented_text})
gg.to_csv("Output_one_dash.csv")

In [ ]:
# augmented_text

In [ ]:
# x=list(tt["tweet"])
# y=list(tt["label"])

In [ ]:
# x.extend(augmented_text)
# y.extend(augmented_label)

In [ ]:
new=pd.DataFrame({"tweet":x,"label":y},)

In [ ]:
# new

In [ ]:
new_train=[]
new_label=[]
for i in range(len(new['tweet'])):
    dic={}
    for elements in new['tweet'][i].split():
        if elements in dic:
            dic[elements] += 1
        else:
            dic.update({elements: 1})
    try: 
        if dic[max(dic, key=dic.get)]>10:
            print(i)
        else:
            new_train.append(new['tweet'][i])
            new_label.append(new['label'][i])
    except:
        print(new['tweet'][i])

In [ ]:
# new_df=pd.DataFrame({'tweet':new_train,'label':new_label})

In [ ]:
# new_df.to_csv("new.csv",index=False)

## Step 2) Train BERT

In [9]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)
import pandas as pd
import numpy as np
from numpy import array
import sklearn
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 47.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 56.9 MB/s eta 0:00:0000:01:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=33eec1243a59759eff4e238eb91af887e4ac75a8ada5e34430bbce10562ea0f0
  Stored in directory: /root/.cache/pip/wheels/b2/a1/b7/0d3b008d0c77cd57332d724b92cf7650b4185b493dc785f00a
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=e8c39e678e7df45133fc62b1127163f85b0ccdeb2d342930d4d9d5a0a6d4c45f
  Stored in directory: /root/.cache/pip/wheels/3d/7c/f1/f87f51d07a2513aa0fb128d

In [10]:
import pandas as pd
train_df=pd.read_csv("/kaggle/working/MahaSent.csv")
gen=pd.read_csv("/kaggle/input/bedata/Output_one_dash.csv")
valid_df=pd.read_csv("/kaggle/working/tweets-valid.csv")
test_df=pd.read_csv("/kaggle/working/tweets-test.csv")

In [11]:
# tt=pd.read_csv("/kaggle/input/bedata/MahaSent/mahasent_train_new_part1.csv")
# import re
# tts=[]
# for i in tt["tweet"]:
#   i= re.sub(r"http\S+", "", i)
#   i= re.sub(r"pic\S+", "", i)
#   i= re.sub(r"#\S+", "", i)
#   i= re.sub(r"lokmat\S+", "", i)
#   i= re.sub(r"@\S+", "", i)
#   i= re.sub(r"fb\S+", "", i)
#   i= re.sub(r"twitter\S+", "", i)
#   i= re.sub(r"bit\S+", "", i)
#   i=i.strip()
#   tts.append(i)
# tt["tweet"]=tts
# tt.to_csv("tweets-train.csv",index=False)

In [12]:
# for i in range(len(gen['tweet'][:10])):
#     print(gen['tweet'][i])

In [13]:
train_df

,Unnamed: 0,Unnamed: 0.1,tweet,label
0,0,0,ज्येष्ठ पत्रकार अनंत दीक्षित यांच्या निधनाचे व...,-1
1,1,1,सर्वोच्च न्यायालयाचे निर्देश डावलून पुणे पोलिस...,-1
2,2,2,उद्धव ठाकरेंनी भाजपासोबत युती करून शिवसैनिकांच...,-1
3,3,3,आपला समाज खूप मोठा आहे. त्यात अनेक घटक अंतर्भू...,1
4,4,4,बलात्काराचा बदला बलात्काराने घेतला पाहिजे हे म...,-1
...,...,...,...,...
6051,6051,12105,काँग्रेस नेते यांनी लिहिलेल्या विधानगाथा या प...,0
6052,6052,12106,"आपल्या अभंग, विचारवाणीतून\nव्यावहारिक सत्य मां...",1
6053,6053,12109,त्याचबरोबर काजू व्यावसायिकांची मागील कालावधीती...,1
6054,6054,12110,सरकारने कर्जमाफी योजना लागू करुन सुमारे २ वर्ष...,-1


In [14]:
train_df=train_df.iloc[:,2:]

In [15]:
gen=pd.DataFrame({'label':gen['label'],'tweet':gen['generated']})

In [16]:
gen

,label,tweet
0,1,"नूतन वर्ष २०१९ सुख, समृद्धीचे आणि यशाचे जाव ो,..."
1,-1,कंगना बाई ही मोदी सरकार प्रमाणेच पूर्णपणे खोटा...
2,0,"घरी रहा, सुरक्षित रहा... आरोग्य सेतू ॲप केले त..."
3,0,"दरम्यान जि गाव प्रकल्पाच्या पाणीसाठ ्याबाबत, प..."
4,-1,आचारसंहिता लागू झाली. सरकारच्या प्रत्येक विभाग...
...,...,...
6051,-1,गत ७० वर्षांच्या कालावधीत पहिल्यांदाच डिझेल पे...
6052,0,"कोल्हापूरची अंबाबाई, कोल्हापूरचा तांब डा - पां..."
6053,-1,"हे प्रधानमंत्री गरीब कल्याण पॅकेज नसून "" प्रधा..."
6054,1,"महिलांना त्यांच्या हक्काची, आत्म निर्भर तेची ज..."


In [17]:
tweet=[]
label=[]
for i in range(len(gen['tweet'])):
    dic={}
    for j in gen['tweet'][i].split():
        dic[j]=0
    for j in gen['tweet'][i].split():
        dic[j]+=1
#     dic=sorted(dic)
    if dic[max(dic,key=dic.get)]<7:
        tweet.append(gen['tweet'][i])
        label.append(gen['label'][i])

In [18]:
gen=pd.DataFrame({'label':label,'tweet':tweet})

In [15]:
# list(train_df['label']).extend(list(gen['label']))

In [19]:
train_df=pd.concat([train_df,gen]).reset_index(drop=True)

In [21]:
train_df['label'] = train_df['label'].replace(1,2)
test_df['label'] = test_df['label'].replace(1,2)
valid_df['label'] = valid_df['label'].replace(1,2)

train_df['label'] = train_df['label'].replace(0,1)
test_df['label'] = test_df['label'].replace(0,1)
valid_df['label'] = valid_df['label'].replace(0,1)

train_df['label'] = train_df['label'].replace(-1,0)
test_df['label'] = test_df['label'].replace(-1,0)
valid_df['label'] = valid_df['label'].replace(-1,0)

In [22]:
%mkdir new_data
%cd new_data
train_df.to_csv('tweets-train.csv')
test_df.to_csv('tweets-test.csv')
valid_df.to_csv('tweets-valid.csv')
%cd ..

/kaggle/working/new_data
/kaggle/working


In [23]:
!pip install --no-cache-dir transformers sentencepiece
!pip install datasets

In [24]:
from datasets import load_dataset
data_files={"train":"tweets-train.csv","test":"tweets-test.csv","validation":"tweets-valid.csv"}
dataset = load_dataset('new_data',data_files=data_files)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/new_data-b5dd43414da02094/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

In [26]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-bert-v2")

def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/marathi-bert-v2", num_labels=3)

from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

Some weights of the model checkpoint at l3cube-pune/marathi-bert-v2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/marat

In [28]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [29]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0, tweet. If Unnamed: 0, tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11714
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2199
  Number of trainable parameters = 237558531
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
import pandas as pd
goe_valid_df=pd.read_csv("/kaggle/input/goemotions/goemotions_subset_val_new.csv")
goe_test_df=pd.read_csv("/kaggle/input/goemotions/goemotions_subset_test_new.csv")

In [ ]:
goe_test_df=goe_test_df.iloc[:,3:]
goe_valid_df=goe_valid_df.iloc[:,3:]

In [ ]:
goe_test_df

In [ ]:
goe_test_df=pd.DataFrame({'label':goe_test_df['three_point'],'tweet':goe_test_df['marathi_text']})
goe_valid_df=pd.DataFrame({'label':goe_valid_df['three_point'],'tweet':goe_valid_df['marathi_text']})

In [ ]:
goe_test_df['tweet'][4]

In [ ]:
# import re
# re.sub(r"\[NAME]+", "", goe_test_df['tweet'][2])
# [नाम]

In [ ]:
tt=goe_test_df.iloc[:,:]
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=re.sub(r"\[NAME]+", "", i)
  i=re.sub(r"\[नाम]+", "",i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("goed_test.csv")

In [ ]:
tt=goe_valid_df.iloc[:,:]
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=re.sub(r"\[NAME]+", "", i)
  i=re.sub(r"\[नाम]+", "",i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("goed_valid.csv")

In [ ]:
goe_valid_df=pd.read_csv("goed_valid.csv")
goe_test_df=pd.read_csv("goed_test.csv")

In [ ]:
goe_test_df=goe_test_df.iloc[:,1:]
goe_valid_df=goe_valid_df.iloc[:,1:]

In [ ]:
goe_test_df=goe_test_df.drop(4294).reset_index(drop=True)

In [ ]:
goe_valid_df=goe_valid_df.drop(1742).reset_index(drop=True)

In [ ]:
# train_df['label'] = train_df['label'].replace(1,2)
goe_test_df['label'] = goe_test_df['label'].replace(1,2)
goe_valid_df['label'] = goe_valid_df['label'].replace(1,2)

# train_df['label'] = train_df['label'].replace(0,1)
goe_test_df['label'] = goe_test_df['label'].replace(0,1)
goe_valid_df['label'] = goe_valid_df['label'].replace(0,1)

# train_df['label'] = train_df['label'].replace(-1,0)
goe_test_df['label'] = goe_test_df['label'].replace(-1,0)
goe_valid_df['label'] = goe_valid_df['label'].replace(-1,0)

In [ ]:
%mkdir new_data2
%cd new_data2
# train_df.to_csv('tweets-train.csv')
goe_test_df.to_csv('tweets-test.csv')
goe_valid_df.to_csv('tweets-valid.csv')
%cd ..

In [ ]:
from datasets import load_dataset
data_files={"test":"tweets-test.csv","validation":"tweets-valid.csv"}
dataset2 = load_dataset('new_data2',data_files=data_files)

In [ ]:
goe_test_df.isnull().sum()

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)

tokenized_datasets2 = dataset2.map(tokenize_function, batched=True)

In [ ]:
# predictions = trainer.predict(tokenized_datasets2["test"])
# preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
predictions = trainer.predict(tokenized_datasets2["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
predictions = trainer.predict(tokenized_datasets2["test"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)